<a href="https://colab.research.google.com/github/UITTrinhQuangTruong/CS114.K21/blob/master/DetectingSarcasm_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras

Using TensorFlow backend.
